In [1]:
from __future__ import print_function, division
import os, sys
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

In [ ]:
dataroot = '/home/abhijit/Downloads/video frames/crowd/'
csv = '/home/abhijit/Downloads/video frames/optical flow/optical flow/crowdgroundtruth.xlsx'
dataroot_2 = '/home/abhijit/Downloads/video frames/optical flow/'

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Loading the Groung Truth for Training


In [ ]:
all_ = pd.read_excel(csv, header = 0)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(all_, test_size=0.3, random_state = 0)
valid_df, test_df = train_test_split(test_df, test_size=.66, random_state = 0)

In [ ]:
train_df = train_df.reset_index()
valid_df = valid_df.reset_index()
test_df = test_df.reset_index()

# DataLoader

In [ ]:
class SyntheticDataset(Dataset):
    

    def __init__(self, csv_file,root_dir,df,root_dir_2, transform=None):
       
        self.landmarks_frame = pd.read_excel(csv_file, header = 0)
        self.read_number = pd.read_excel(csv_file,header = 1)
        self.root_dir = root_dir
        self.root_dir_2 = root_dir_2
        self.transform = transform
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.df.iloc[idx]['Frame1']
        number_left = self.df.iloc[idx,3]
        number_right = self.df.iloc[idx,4]
        number_combined = self.df.iloc[idx,5]
        #number_all = self.landmarks_frame['left_to_right'][idx:idx]
        #number_all = 0
        number_all = np.array([number_left,number_right, number_combined])
        
        ix = ['scene1','scene2','scene3','scene4','scene5','scene6','scene7','scene8','scene9', 'scene10','scene11','scene12','scene13','scene14','scene15']
        jx = ['scene10','scene11','scene12','scene13','scene14','scene15']

#img_name = 'scene6medcam1frame780'
        i = ''
        x = 0
        for j in jx:
            if j in img_name:
                x = 1
                break
        i = j
        if x==0:
            for i in ix:
                if i in img_name:
                    break
        
        dense = ['low', 'med', 'high']
        cam = ['cam1', 'cam2', 'cam3']
        occlu = ['', 'occlu']   

        for den in dense:
            if den in img_name:
                break
        for c in cam:
            if c in img_name:
                break
            
        if 'occlu' in img_name:
            occ = 'occlu'
        else:
            occ= ''
        
        #image = io.imread(self.root_dir+'scene' + str(i)+'/'+'scene'+str(i)+den+c+occ+'/'+img_name )
        #img = self.root_dir+ str(i)+'/'+str(i)+den+c+occ+'/'+img_name+'.jpg'
        img = self.root_dir+ '/'+img_name+'.png'
        optical_flow = self.root_dir_2 + str(i)+'/'+str(i)+den+c+occ+'/'+img_name+'.png'
        #image = io.imread(img)
        #image = Image.fromarray(image)
        
        image = Image.open(img).convert('RGB')
        op_flow = io.imread(optical_flow)
        op_flow = Image.fromarray(op_flow)
        origi = image
        image = self.transform(image)
        op_flow = self.transform(op_flow)
        
        every_thing = {'image': image,
                        'img_root':img,
                       'optical_flow':op_flow,
                      'left_to_right': number_left,
                      'right_to_left': number_right,
                      'combined':number_combined,
                      'all':number_all
                }
        

        return every_thing

# Test Dataset Locations

In [ ]:
dataroot3 = '/home/abhijit/Documents/RealData/removeback/1stvideo'
dataroot4 = '/home/abhijit/Documents/RealData/removebackoptical/1stvideo'

df4 = '/home/abhijit/Documents/RealData/ground.xlsx'
df4 = pd.read_excel(df4, header = 0)

In [ ]:
df2 = pd.read_excel(df2, header = 0)
df3 = pd.read_excel(df3, header = 0)

In [ ]:
df = pd.concat([df2,df3,df4],axis = 0)

In [ ]:
real_dataset_test =  SyntheticDataset2( dataroot1,dataroot2,test_df_2,
                                    transform = transforms.Compose([ transforms.Resize([227,227]),  transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))
real_loader_test = torch.utils.data.DataLoader(real_dataset_test, batch_size= 1, shuffle=True)

# AlexNet as CNN Backbone

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import torch.utils.model_zoo


class AlexNet_modified(nn.Module):
    def __init__(self, alexnet, n_class=1000):
        super(AlexNet_modified, self).__init__()
        # conv1
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        # self.bn1 = nn.BatchNorm2d(64, momentum=0.1)
        self.relu1 = nn.ReLU(inplace=True)
        # LRN(local_size=5, alpha=0.0001, beta=0.75)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)

        # conv2
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        # self.bn2 = nn.BatchNorm2d(192, momentum=0.1)
        self.relu2 = nn.ReLU(inplace=True)
        # LRN(local_size=5, alpha=0.0001, beta=0.75)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)

        # conv3
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        # self.bn3 = nn.BatchNorm2d(384, momentum=0.1)
        self.relu3 = nn.ReLU(inplace=True)

        # conv4
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256, momentum=0.1)
        self.relu4 = nn.ReLU(inplace=True)

        # conv5
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        # self.bn5 = nn.BatchNorm2d(256, momentum=0.1)
        self.relu5 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=3, stride=2)
        # self.pool5 = nn.AdaptiveAvgPool2d(6)
        self.drop5 = nn.Dropout()

        # fc6
        self.fc6 = nn.Linear(256 * 6 * 6, 4096)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout()

        # fc7
        self.fc7 = nn.Linear(4096, 4096)
        self.relu7 = nn.ReLU(inplace=True)

        # fc8 this is the extra layer
        self.fc8 = nn.Linear(4096*2,4096)
        self.fc9 = nn.Linear(4096,1024)
        self.fc10 = nn.Linear(1024, 512)
        self.fc11 = nn.Linear(512, n_class)
        self.copy_params_from_alexnet(alexnet)

    def sequence(self, x):
        h = x
        # h = self.relu1(self.bn1(self.conv1(h)))
        h = self.relu1(self.conv1(h))
        h = self.pool1(h)

        # h = self.relu2(self.bn2(self.conv2(h)))
        h = self.relu2(self.conv2(h))
        h = self.pool2(h)

        # h = self.relu3(self.bn3(self.conv3(h)))
        h = self.relu3(self.conv3(h))

        # h = self.relu4(self.bn4(self.conv4(h)))
        h = self.relu4(self.conv4(h))

        # h = self.relu5(self.bn5(self.conv5(h)))
        h = self.relu5(self.conv5(h))
        h = self.pool5(h)
        h = self.drop5(h)

        h = h.view(-1, 256 * 6 * 6)
        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        
        
        

        
        return h

    def forward(self, x1, x2):
        # branch 1
        h1 = self.sequence(x1)

        # branch 2
        h2 = self.sequence(x2)

        # merge two branches

        group = torch.cat((h1, h2), 1)
        
        h = self.relu7(self.fc8(group))
        h = self.relu7(self.fc9(h))    
        h = self.relu7(self.fc10(h))
        o = self.fc11(h)
        return o

    def copy_params_from_alexnet(self, alexnet):
        features = [
            self.conv1, self.relu1,
            self.pool1,
            self.conv2, self.relu2,
            self.pool2,
            self.conv3, self.relu3,
            self.conv4, self.relu4,
            self.conv5, self.relu5,
            self.pool5,
        ]
        for l1, l2 in zip(alexnet.features, features):
            if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
                assert l1.weight.size() == l2.weight.size()
                assert l1.bias.size() == l2.bias.size()
                l2.weight.data = l1.weight.data
                l2.bias.data = l1.bias.data
        for i, name in zip([1, 4], ['fc6', 'fc7']):
            l1 = alexnet.classifier[i]
            l2 = getattr(self, name)
            l2.weight.data = l1.weight.data.view(l2.weight.size())
            l2.bias.data = l1.bias.data.view(l2.bias.size())


def main(paras):
    if paras == 'alexnet':
        model = models.alexnet(pretrained=True)
        net_modified = AlexNet_modified(model, 3)

    return net_modified



In [ ]:
use_gpu = torch.cuda.is_available()
import torch.backends.cudnn as cudnn

In [ ]:
model = main('alexnet')
model.cuda()

# Optimization Parameters

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
epoch = 50

lr = 1e-5

loss = nn.MSELoss()

optimizerD = optim.Adam(b.parameters(), lr= 1e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizerD, step_size=10, gamma=0.1)


from torch.autograd import Variable

# Training and Validation

In [ ]:
epochs = 50


train_min = 5000
valid_min = 5000
train_error_all = list()
for epoch in range(epochs):
    
    train_error = 0
    valid_error= 0
    b.train()
    for i, data in enumerate(real_loader,0):
        
    

        image = data['image']
        optical_flow = data['optical_flow']
        
        
        image = Variable(image.cuda())
        optical_flow = Variable(optical_flow.cuda())
        target = Variable(data['all'].float()).cuda()
        
        
        
     
    
        
        c = b(image, optical_flow)
        

        error = loss(c, target)
        #error = torch.tensor(error, requires_grad=True)
        #error = torch.tensor(.5)
        #error = torch.tensor(error, dtype = 'float')
        #error = error.float()
        train_error +=error
        #if (i % 15) == 0:
            #print(c)
            #print(data['all'])
        
            #print('\n \n')    
            
            #print(error)
        optimizerD.zero_grad()
        error.backward()
        optimizerD.step()
        #print(5)
        #print(d)
        #print('\n \n')
        #print(target)
        
    train_error = train_error/(i+1)
    if train_error < train_min:
        train_min =  train_error
        
    train_error_all.append(train_error)
    with torch.set_grad_enabled(False):

        b.eval()
        for i, data_sample in enumerate(real_loader_test):
            image = data_sample['image']
            optical_flow = data_sample['optical_flow']
        
        
            image = Variable(image.cuda())
            optical_flow = Variable(optical_flow.cuda())
            target = Variable(data_sample['all'].float()).cuda()
        
        
        
     
    
        
            c = model(image, optical_flow)
            
            #if epoch % 20 ==0:
                
             #   print(data_sample['all'])
              #  print('\n')
               # print(c)
                #print("\n \n")
        

            error = loss(c, target)
        
            valid_error += error
        
        valid_error = valid_error/(i+1)

    
        if valid_error < valid_min:
            valid_min =  valid_error
    
        
            torch.save({ 'epoch':epoch, 'model':b, 'state_dict': model.state_dict(),
                                   'optimizer_state_dict': optimizerD.state_dict(),
                'loss': train_error}, '/home/abhijit/Documents/saved_models/b_final_finetune_onreal')
        
    print('train_error: %f   valid_error: %f' %(train_error, valid_error))
    
    exp_lr_scheduler.step()

# Performing the Test

In [ ]:
a = 1


test_error_all = list()
ground_truth_all = list()

test_error_all_ucsd = list()
ground_all_ucsd = list()
test_error_all_bolei = list()
ground_truth_all_bolei = list()
test_error = 0
test_error_all_my = list()

ground_truth_my = list()
ucsd = 0
bolei = 0
my_ = 0
if a ==1: 
    with torch.set_grad_enabled(False):

        b.eval()
        for i, data_sample in enumerate(real_loader_test):
            image = data_sample['image']
            optical_flow = data_sample['optical_flow']
        
        
            image = Variable(image.cuda())
            optical_flow = Variable(optical_flow.cuda())
            target = Variable(data_sample['all'].float()).cuda()
        
        
        
     
    
        
            c = b(image, optical_flow)
            
            
            test_error_all.append(c)
            ground_truth_all.append(target)
            
            #print(data_sample['type'])
            
            if data_sample['type'][0] == 'bolei':
                test_error_all_bolei.append(c)
              #
                ground_truth_all_bolei.append(target)
                bolei +=1
                #print(bolei)
                
            elif data_sample['type'][0]=='mydata':
                
                test_error_all_my.append(c)

                ground_truth_my.append(target)
                my_ +=1
                
            else:   
                
                test_error_all_ucsd.append(c)
                ground_all_ucsd.append(target)
                ucsd +=1
            
            #a = 1
            if (i%25)  :
                
                trans = transforms.ToPILImage()
                img = data_sample['img_root']
                image = io.imread(img[0])
        #plt.imshow(image)
        #print(data['image'].shape)
        #image = data['optical_flow'][0]
        #plt.imshow(image)
                #plt.imshow(image.permute(1, 2, 0)  )
                #images = image.cpu()
                #images = io.imread(images[0])
                #images = images[0].permute(1,2,0)
                #images = images[0]
                #unnorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                #images = unnorm(images)
                #images = images/2 + .416
                #images[0] = images[0]*.229 +.486
                #images[1]= images[1]*.224+.456
               # images[2] = images[2]*.225 +.406
                #im = Image.open(images)
                #im = transforms.ToPILImage()(images[0]).convert("RGB")
                #display(im)
                #plt.imshow(trans(images[0])).convert('RGB')
                #plt.show()
                #plt.figure(10,10)
                from matplotlib.pyplot import figure
                #figure(10,10)
                plt.figure(figsize=(6,6))
                plt.imshow(image, cmap = 'gray')
                plt.show()
                print('Ground Truth \n')
                print(data_sample['all'][0])
                print('\n')
                print('Predicted \n')
                print(c[0])
                print("\n \n")
            

            #error = loss(c, target)
            #test_error_all.append(c)
            #ground_all.append(target)
        
            #test_error += error
        
        #test_error = test_error/(i+1)
